In [20]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [69]:
len(food["company"].unique())

193

In [75]:
idx = food.groupby("company")["pounds"].transform(max) == food["pounds"]
food[idx][~food[idx].duplicated("company")]["weekday"]

52         Monday
53         Monday
100      Thursday
103        Friday
124      Thursday
151        Friday
216        Friday
224        Friday
279     Wednesday
300      Thursday
307        Friday
358      Thursday
440        Friday
445        Monday
477       Tuesday
526      Thursday
528        Friday
582     Wednesday
693      Thursday
695        Friday
757     Wednesday
783      Thursday
784      Thursday
861        Friday
888        Monday
923      Thursday
926     Wednesday
928     Wednesday
942        Friday
952        Friday
          ...    
4217       Monday
4237    Wednesday
4268    Wednesday
4296       Friday
4302       Friday
4308       Friday
4360    Wednesday
4390     Thursday
4433     Thursday
4435       Friday
4437       Friday
4479      Tuesday
4492    Wednesday
4549       Friday
4554       Friday
4563     Thursday
4585       Monday
4604      Tuesday
4610       Monday
4614    Wednesday
4655       Friday
4677       Friday
4679       Friday
4682       Friday
4684     T

In [181]:
food = pd.read_csv("food_cleaned.csv")
food["date"] = pd.to_datetime(food["date"])
company_weeks = food.groupby("company")["date"].agg(lambda x: int((max(x) - min(x))/np.timedelta64(1, "W")))+ 1
company_tot_donation = food.groupby("company")["pounds"].agg(np.sum)
company_num_donation = food.groupby("company")["pounds"].count()
company_one_donation = (company_num_donation == 1).astype("int")
company_pound_per_week = company_tot_donation/company_weeks
company_avg_donation = company_tot_donation/company_num_donation
company_max_donation = food.groupby("company")["pounds"].agg(np.max)
company_med_donation = food.groupby("company")["pounds"].agg(np.median)
company_size = food.groupby("company")["size"].first()
company_type = food.groupby("company")["type"].first()
company_city = food.groupby("company")["city"].first()
company_state = food.groupby("company")["state"].first()
company_most_donation_weekday = food.groupby("company")["weekday"].agg(lambda x: np.argmax(x.value_counts()))
company_most_donation_weekday_frac = food.groupby("company")["weekday"].agg(lambda x: np.max(x.value_counts()))/company_num_donation
company_max_weekday = food.loc[food.reset_index().groupby("company")["pounds"].idxmax()]["weekday"]
company_max_weekday.index = company_weeks.index
company_most_donation_month = food.groupby("company")["month"].agg(lambda x: np.argmax(x.value_counts()))
company_most_donation_month_frac = food.groupby("company")["month"].agg(lambda x: np.max(x.value_counts()))/company_num_donation
company_start_month = food.groupby("company")["month"].agg(np.min)
company = pd.DataFrame({#"tot_donation": company_tot_donation, 
                       "num_donation": company_num_donation, 
                        "num_weeks": company_weeks, 
                       #"avg_donation": company_avg_donation, 
                        "week_donation": company_pound_per_week,
                       #"max_donation": company_max_donation, 
                        #"med_donation": company_med_donation,
                        "size": company_size, "type": company_type,
                       "popular_weekday": company_most_donation_weekday,
                        "popular_month": company_most_donation_month,
                       "popular_weekday_frac": company_most_donation_weekday_frac,
                        "popular_month_frac": company_most_donation_month_frac,
                       "max_pound_weekday": company_max_weekday, "start_month": company_start_month,
                        "one_donation": company_one_donation, "state": company_state
                       })
company["top_contributor"] = company["num_donation"].apply(lambda x: 1 if x >= 80
                                                          else 0)
company.loc[company.index == "Imgur", "popular_weekday"] = "Friday"
company = company[~(company.index == "Curiodysse")]

c:\users\jordan fan\anaconda3\envs\toxic\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [182]:
def ohe_max_pound_weekday(data):
    cats = data["max_pound_weekday"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data['max_pound_weekday'] = data['max_pound_weekday'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='max_weekday',
                    columns=['max_pound_weekday'], 
                    drop_first=True)
    return data

def ohe_popular_month(data):
    cats = data["popular_month"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data["popular_month"] = data['popular_month'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='pop_month',
                    columns=['popular_month'], 
                    drop_first=True)
    return data
    
def ohe_popular_weekday(data):
    cats = data["popular_weekday"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data['popular_weekday'] = data['popular_weekday'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='pop_weekday',
                    columns=['popular_weekday'], 
                    drop_first=True)
    return data

def ohe_size(data): 
    cats = data["size"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data['size'] = data['size'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='size',
                    columns=['size'], 
                    drop_first=True)
    return data

def ohe_start_month(data):
    cats = data["start_month"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data['start_month'] = data['start_month'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='start_month',
                    columns=['start_month'], 
                    drop_first=True)
    return data

def ohe_state(data):
    cats = data["state"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data['state'] = data['state'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='state',
                    columns=['state'], 
                    drop_first=True)
    return data

def ohe_type(data):
    cats = data["type"].unique().tolist()
    cat_type = CategoricalDtype(categories=cats)
    data['type'] = data['type'].astype(cat_type)
    data = pd.get_dummies(data,
                    prefix='type',
                    columns=['type'], 
                    drop_first=True)
    return data

def process_data(data):
    data = (
        data
        # Clean Data
        .pipe(ohe_max_pound_weekday)
        .pipe(ohe_popular_month)
        .pipe(ohe_popular_weekday)
        .pipe(ohe_size)
        #.pipe(ohe_start_month)
        #.pipe(ohe_state)
        .pipe(ohe_type)
    )
    return data
process_data(company).to_csv("avg_week_donation.csv", sep = ",")


In [183]:
company.to_csv("avg_week_donation_raw.csv", sep = ",")

In [155]:
company_processed = process_data(company).drop(["state", "start_month", "num_donation", "num_weeks", "pop_month_2", "pop_month_3", "pop_month_4", "pop_month_5", "pop_month_6", "pop_month_7", "pop_month_8", "pop_month_10"], axis = 1)
#poly = PolynomialFeatures(degree = 3)
#X_ = poly.fit_transform(company_processed)
#predict_ = poly.fit_transform(company_pound_per_week)

In [151]:
X_ = poly.fit_transform(company_processed)

In [157]:
company_processed.to_csv("avg_week_donation.csv")